In [1]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!kaggle datasets download -d salader/dogs-vs-cats

Dataset URL: https://www.kaggle.com/datasets/salader/dogs-vs-cats
License(s): unknown


In [2]:
!pip install torchmetrics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 961.5/961.5 kB 37.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 91.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 69.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 55.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 74.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstall

In [3]:
import zipfile
zip_ref = zipfile.ZipFile('dogs-vs-cats.zip', 'r')
zip_ref.extractall()
zip_ref.close()

In [4]:
import torch
from torch.utils.data import  DataLoader
from torchvision import datasets, transforms
from torch import nn
from torchmetrics.classification  import BinaryAccuracy

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [6]:
base_transformation = [
    transforms.Resize((288, 288)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
]

In [7]:
train_transformation = transforms.Compose([
    transforms.RandomHorizontalFlip(p=.5),
    transforms.RandomVerticalFlip(p=.5),
    transforms.RandomRotation(degrees=20),
    *base_transformation
])

test_transformation = transforms.Compose([
    *base_transformation
])

In [8]:
train = datasets.ImageFolder("/content/train", transform=train_transformation)
test = datasets.ImageFolder("/content/test", transform=test_transformation)
print(train.class_to_idx)

{'cats': 0, 'dogs': 1}


In [9]:
torch.manual_seed(42)
data_loader = DataLoader(train, batch_size=32, shuffle=True)

In [16]:
class CNN(nn.Module):
  def __init__(self):
    super().__init__()

    self.feature = nn.Sequential(
      #1
      nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding='valid'),
      nn.ReLU(),
      nn.MaxPool2d(kernel_size=2, stride=2),
      #2
      nn.Conv2d(in_channels = 32, out_channels = 64, kernel_size=3, padding='valid'),
      nn.ReLU(),
      nn.MaxPool2d(kernel_size=2, stride=2),
      #3
      nn.Conv2d(in_channels = 64, out_channels = 128, kernel_size=3, padding='valid'),
      nn.ReLU(),
      nn.MaxPool2d(kernel_size=2, stride=2)
    )

    self.classifier = nn.Sequential(
      nn.Flatten(),
      #4
      nn.Linear(in_features = 128*34*34 , out_features=128),
      nn.ReLU(),
      nn.Dropout(p=0.3),
      #5
      nn.Linear(in_features=128, out_features=64),
      nn.ReLU(),
      nn.Dropout(p=0.3),
      #6
      nn.Linear(in_features=64, out_features=1),
      nn.Sigmoid()
    )

  def forward(self, x):
    x = self.feature(x)
    x = self.classifier(x)
    return x

In [17]:
def train(epochs, train_loader, model, criteration, optimizer, accuracy):
  loss_list = []
  accuracy_list = []
  for epoch in range(epochs):
    loss_batch = 0
    acc_sum = 0
    num_batches = 0
    for X, y in train_loader:
      X = X.to(device)
      y = y.to(device)
      y = y.view(-1, 1).float()
      #forward pass
      y_pred = model(X)
      #loss calculation
      loss = criteration(y_pred, y)
      #reset optimizer
      optimizer.zero_grad()
      #backpropagations
      loss.backward()
      #upgrade grad
      optimizer.step()

      loss_batch += loss.item()

      acc_sum += accuracy(y_pred, y).item()
      num_batches += 1

    loss_list.append(loss_batch/len(train_loader))
    avg_acc = acc_sum / num_batches
    accuracy_list.append(avg_acc)

    print(f"Epoch: {epoch} Loss: {loss_batch/len(train_loader)} Accuracy: {avg_acc}")

  return loss, accuracy

In [18]:
model = CNN().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.BCELoss()
accuracy = BinaryAccuracy(threshold=0.5).to(device)
train(
    10,
    data_loader,
    model,
    nn.BCELoss(),
    optimizer,
    accuracy
    )

Epoch: 0 Loss: 0.6404064985752106 Accuracy: 0.61765
Epoch: 1 Loss: 0.5753742481708527 Accuracy: 0.69765
Epoch: 2 Loss: 0.5275131682872772 Accuracy: 0.7379
Epoch: 3 Loss: 0.49981240248680114 Accuracy: 0.7586
Epoch: 4 Loss: 0.48169421112537386 Accuracy: 0.77185
Epoch: 5 Loss: 0.45824241967201235 Accuracy: 0.78575
Epoch: 6 Loss: 0.441744539642334 Accuracy: 0.7987
Epoch: 7 Loss: 0.42265775001049044 Accuracy: 0.8094
Epoch: 8 Loss: 0.40931461234092714 Accuracy: 0.81685
Epoch: 9 Loss: 0.4037194330215454 Accuracy: 0.82395


(tensor(0.2736, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>),
 BinaryAccuracy())

In [19]:
data_loader_test = DataLoader(test, batch_size=32, shuffle=False)

total_loss = 0
total_correct = 0
total_samples = 0

with torch.no_grad():
    for X_valid, y_valid in data_loader_test:
        X_valid = X_valid.to(device)
        y_valid = y_valid.to(device)
        y_valid = y_valid.view(-1, 1).float()

        pred = model(X_valid)
        loss = criterion(pred, y_valid)

        preds = (pred >= 0.5).int()

        total_loss += loss.item() * len(X_valid)
        total_correct += (preds == y_valid).sum().item()
        total_samples += len(X_valid)

avg_loss = total_loss / total_samples
avg_accuracy = total_correct / total_samples

print("###############################")
print(f'Average Validation Loss: {avg_loss:.4f} | Average Accuracy: {avg_accuracy:.4f}')
print("###############################")

###############################
Average Validation Loss: 0.4150 | Average Accuracy: 0.8190
###############################


In [20]:
torch.save(model.state_dict(), 'dogs_vs_cats_model.pth')